# Notebook initial pipeline and signal processing 

## initial thought process on steps to follow with pulling in data
- pull in metadata and convert from EDF to pd: 
    - ``|user_id|device_id|disease|personal_information|``
- access respective user_id data folder
- pull in data from prespective device_id EDF data file 
    - ``|device_id|type|time|value|``
- convert EDF to pandas dataframe
- merge required files together (for respective calculation)

In [2]:
import sys
import getpass
from pathlib import Path

import pandas as pd
import numpy as np
import scipy as sp

import matplotlib.pyplot as plt

import pyedflib

In [44]:
user = getpass.getuser()
data_path = '/Users/{}/Desktop/GitHub/fydp-data-science/sample_files/'.format(user)

# hasn't been created yet -- either develop own or get from ONDRI
metadata = data_path + 'metadata.edf'

In [47]:
def convert_data_file(data_path, filename):    
    # pull data out of the EDF file
    file = pyedflib.EdfReader(data_path + filename)
    n = file.signals_in_file
    signal_labels = file.getSignalLabels()
    sigbufs = np.zeros((n, file.getNSamples()[0]))

    for i in np.arange(n):
        sigbufs[i,:] = file.readSignal(i)
        
    # close file
    file.close()
        
    # create dataframe
    df = pd.DataFrame(columns=signal_labels)
    for i in np.arange(0,len(signal_labels)):
        label = signal_labels[i]
        df[label] = sigbufs[i]
        
    return df

In [51]:
convert_data_file(data_path, 'dX.edf')

,dX
0,-2.000000
1,-1.007812
2,-1.001953
3,-1.001953
4,-1.004883
...,...
6914555,-1.035156
6914556,-1.009766
6914557,-1.005859
6914558,-1.019531
